<a href="https://colab.research.google.com/github/alex-pakalniskis/gisc606-spring2023/blob/main/lab3/GISC606_L3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: Lidar data products (10 points)
*Adaptation of [Chapter 5: Raster Data Processing in Python](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/) from [Use Data for Earth and Environmental Science in Open Source Python Textbook](https://www.earthdatascience.org/courses/use-data-open-source-python/) by [Earth Lab at University of Colorado, Boulder](http://www.colorado.edu/earthlab)*

## Objectives
* Practice using the [Linux command line](https://ubuntu.com/tutorials/command-line-for-beginners#1-overview) 
* Calculate a canopy height model (CHM) with Python on [LiDAR data](https://earthpy.readthedocs.io/en/latest/earthpy-data-subsets.html#colorado-flood)
    * Data were collected by National Ecological Observatory Network (NEON) before and after the [2013 Colorado floods](https://en.wikipedia.org/wiki/2013_Colorado_floods) 





## Part 0: Install Python dependencies in Google Colaboratory
We're using some non-standard GIS libraries that aren't pre-installed in Colaboratory, so let's install them before we begin our analysis.
1. Use [`pip`](https://pypi.org/project/pip/), the Python package installer, to install [`earthpy`](https://earthpy.readthedocs.io/en/latest/) and [`rioxarray`](https://corteva.github.io/rioxarray/stable/).

In [ ]:
!pip install earthpy rioxarray



## Part 1: Download Raster Data

1. Download the lab data using [`wget`](https://www.gnu.org/software/wget/). We're using the data linked at the "Download Colorado Flood Teaching Data Subset data" button found here: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/



In [ ]:
!wget https://ndownloader.figshare.com/files/12395030

2. Display directory contents after data download with [`ls`](https://man7.org/linux/man-pages/man1/ls.1.html).

In [ ]:
!ls

3. Display downloaded data file type with [`file`](https://man7.org/linux/man-pages/man1/file.1.html)

In [ ]:
!file 12395030

4. Unzip the data with [`unzip`](https://manpages.ubuntu.com/manpages/focal/man1/unzip.1.html)

In [ ]:
!unzip 12395030

5. Display directory contents after unzipping (use `ls`)

In [ ]:
!ls

In [ ]:
!ls discharge

In [ ]:
!ls precipitation

In [ ]:
!ls spatial

In [ ]:
!ls spatial/boulder-leehill-rd/

In [ ]:
!ls spatial/boulder-lidar-points/

6. Display directory contents as a "tree" with [`tree`](https://linux.die.net/man/1/tree)

In [ ]:
!apt-get install tree
!tree .

## Part 2: Subtract Rasters & Export Geotiffs
1. Import `python` dependencies

In [ ]:
import os #https://docs.python.org/3/library/os.html
import numpy as np #https://numpy.org/
import matplotlib.pyplot as plt #https://matplotlib.org/stable/tutorials/introductory/pyplot.html
import seaborn as sns #https://seaborn.pydata.org/
import rioxarray as rxr #https://corteva.github.io/rioxarray/stable/
import earthpy as et #https://earthpy.readthedocs.io/en/latest/

# Prettier plotting with seaborn
sns.set_theme(font_scale=1.5, style="whitegrid") #https://seaborn.pydata.org/generated/seaborn.set_theme.html#seaborn.set_theme

2. Open and plot the lidar DEM metadata and data

In [ ]:
# Define relative path to file
lidar_dem_path = os.path.join("spatial",
                              "boulder-leehill-rd",
                              "pre-flood",
                              "lidar",
                              "pre_DTM.tif") #https://docs.python.org/3/library/os.path.html#os.path.join

# Open lidar dem
lidar_dem_xr = rxr.open_rasterio(lidar_dem_path, masked=True).squeeze() #https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray.open_rasterio , https://xarray.pydata.org/en/v2023.03.0/generated/xarray.Dataset.squeeze.html#xarray-dataset-squeeze
lidar_dem_xr

In [ ]:
# Plot the data
f, ax = plt.subplots(figsize=(10, 5)) #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
lidar_dem_xr.plot(cmap="Greens") #https://docs.xarray.dev/en/stable/generated/xarray.DataArray.plot.html
ax.set(title="DEM for Lee Hill Road Pre-Flood") #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set.html
ax.set_axis_off() #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_axis_off.html
plt.show() #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.show.html

3. Open and plot the lidar DSM metadata and data

In [ ]:
# Define relative path to file
lidar_dsm_path = os.path.join("spatial",
                              "boulder-leehill-rd",
                              "pre-flood",
                              "lidar",
                              "pre_DSM.tif")

# Open lidar dem
lidar_dsm_xr = rxr.open_rasterio(lidar_dsm_path, masked=True).squeeze()
lidar_dsm_xr

In [ ]:
# Plot the data
f, ax = plt.subplots(figsize=(10, 5))
lidar_dsm_xr.plot(cmap="Greens")
ax.set(title="DSM for Lee Hill Road Pre-Flood")
ax.set_axis_off()
plt.show()

4. Calculate the canopy height model (CHM) and plot the result

$CHM = DSM - DEM$

In [ ]:
# First check that the rasters cover the same spatial extent and have the same spatial resolution
# Are the bounds the same?
print("Is the spatial extent the same?",
      lidar_dem_xr.rio.bounds() == lidar_dsm_xr.rio.bounds()) #https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray.rioxarray.XRasterBase.bounds

# Is the resolution the same ??
print("Is the resolution the same?",
      lidar_dem_xr.rio.resolution() == lidar_dsm_xr.rio.resolution()) #https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray.rioxarray.XRasterBase.resolution

In [ ]:
# Calculate canopy height model
lidar_chm_xr = lidar_dsm_xr - lidar_dem_xr

# Plot the data
f, ax = plt.subplots(figsize=(10, 5))
lidar_chm_xr.plot(cmap="Greens")
ax.set(title="Canopy Height Model for Lee Hill Road Pre-Flood")
ax.set_axis_off()
plt.show()

5. Investigate the CHM data

In [ ]:
# Plot a histogram of the CHM data
lidar_chm_xr.plot.hist() #https://docs.xarray.dev/en/stable/generated/xarray.plot.hist.html
plt.show()

In [ ]:
# Check the minimum and maximum CHM values
print('CHM minimum value: ', np.nanmin(lidar_chm_xr)) #https://numpy.org/doc/stable/reference/generated/numpy.nanmin.html
print('CHM max value: ', np.nanmax(lidar_chm_xr)) #https://numpy.org/doc/stable/reference/generated/numpy.nanmax.html


6. Save CHM data as a Geotiff

In [ ]:
data_path = os.path.join("spatial",
                         "outputs")

if os.path.exists(data_path): # https://docs.python.org/3/library/os.path.html#os.path.exists
    print("The directory", data_path, "exists!")
else:
    os.makedirs(data_path) #https://docs.python.org/3/library/os.html#os.makedirs

# Make sure that your output data has a crs & no data value defined
print("The crs is", lidar_chm_xr.rio.crs) #https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.rioxarray.XRasterBase.crs
print("The no data value is", lidar_chm_xr.rio.nodata) #https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.raster_array.RasterArray.nodata

pre_chm_data_path = os.path.join(data_path, "pre-flood-chm.tif")
print(f"CHM data saved here: {pre_chm_data_path}")


# Export data to geotiff
lidar_chm_xr.rio.to_raster(pre_chm_data_path) #https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.raster_array.RasterArray.to_raster


7. Open the CHM Geotiff and plot it

In [ ]:
lidar_chm_xr_test = rxr.open_rasterio(pre_chm_data_path, masked=True).squeeze()
lidar_chm_xr_test

In [ ]:
# Plot the data
f, ax = plt.subplots(figsize=(10, 5))
lidar_chm_xr_test.plot(cmap="Greens")
ax.set(title="Canopy Height Model (Geotiff) for Lee Hill Road Pre-Flood")
ax.set_axis_off()
plt.show()

## Part 3: Manually Reclassify Raster Data
1. Import some more `python` dependencies

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm #https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.ListedColormap.html , https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.BoundaryNorm.html
import earthpy.plot as ep #https://earthpy.readthedocs.io/en/latest/api/earthpy.plot.html
import xarray as xr #https://docs.xarray.dev/en/stable/

2. Display some raster summary statistics

In [ ]:
f, ax = plt.subplots()
lidar_chm_xr.plot.hist(color="purple")
ax.set(title="Distribution of Raster Cell Values in the CHM Data",
       xlabel="Height (m)",
       ylabel="Number of Pixels")
plt.show()

In [ ]:
# Histogram with custom breaks
f, ax = plt.subplots()
lidar_chm_xr.plot.hist(color="purple",
                        bins=[0, 5, 10, 15, 20, 30])
ax.set(title="Histogram with Custom Breaks",
       xlabel="Height (m)",
       ylabel="Number of Pixels")

plt.show()

* 0 - 2 = no trees
* 2 - 7 = short trees
* 7 - 12 = medium trees
* greater than 12 = tall trees

In [ ]:
# Histogram with custom breaks
f, ax = plt.subplots()

lidar_chm_xr.plot.hist(
    color='purple',
    bins=[0, 2, 7, 12, 30])
ax.set(title="Histogram with Custom Breaks",
       xlabel="Height (m)",
       ylabel="Number of Pixels")

plt.show()

3. Map raster values to new values (reclassify)

* No trees: (0m - 2m tall) = NA
* Short trees: (2m - 7m tall) = 1
* Medium trees: (7m - 12m tall) = 2
* Tall trees: (> 12m tall) = 3

> Notice in the list above that you set cells with a value between 0 and 2 meters to NA or nodata value. This means you are assuming that there are no trees in those locations

* 0 - 2 meters -> 1
* 2 - 7 meters -> 2 (short trees)
* 7 - 12 meters -> 3 (medium trees)
* greater than 12 or 12 - Inf -> 4 (tall trees)




In [ ]:
data_min_value = np.nanmin(lidar_chm_xr)
data_max_value = np.nanmax(lidar_chm_xr)

class_bins = [-np.inf, 2, 7, 12, np.inf]

lidar_chm_xr_class = xr.apply_ufunc(np.digitize,
                                     lidar_chm_xr,
                                     class_bins) #https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html

im = lidar_chm_xr_class.plot.imshow() #https://docs.xarray.dev/en/stable/generated/xarray.plot.imshow.html
ax.set_axis_off()


Let's try to make it a bit nicer looking by masking the no-data class

In [ ]:
pre_lidar_chm_class_ma = lidar_chm_xr_class.where(lidar_chm_xr_class != 5) #https://docs.xarray.dev/en/stable/generated/xarray.where.html

# Plot newly classified and masked raster
f, ax = plt.subplots(figsize=(10,5))
pre_lidar_chm_class_ma.plot.imshow()
ax.set(title="Classified Plot With a Colorbar")

ax.set_axis_off()
plt.show()


Let's try to improve the visualization a bit more by using specific colors.

In [ ]:
# Plot data using nicer colors
colors = ['linen', 'lightgreen', 'darkgreen', 'maroon']
class_bins = [.5, 1.5, 2.5, 3.5, 4.5]
cmap = ListedColormap(colors)
norm = BoundaryNorm(class_bins, 
                    len(colors))

# Plot newly classified and masked raster
f, ax = plt.subplots(figsize=(10, 5))
pre_lidar_chm_class_ma.plot.imshow(cmap=cmap,
                                   norm=norm)
ax.set(title="Classified Plot With a Colorbar and Custom Colormap (cmap)")
ax.set_axis_off()
plt.show()

And finally let's add a legend

In [ ]:
# Create a list of labels to use for your legend
height_class_labels = ["Short trees",
                       "Medium trees",
                       "Tall trees",
                       "Really tall trees"]

# Create a colormap from a list of colors
colors = ['linen',
          'lightgreen',
          'darkgreen',
          'maroon']

cmap = ListedColormap(colors)

class_bins = [.5, 1.5, 2.5, 3.5, 4.5]
norm = BoundaryNorm(class_bins,
                    len(colors))

# Plot newly classified and masked raster
f, ax = plt.subplots(figsize=(10, 5))
im = pre_lidar_chm_class_ma.plot.imshow(cmap=cmap,
                                        norm=norm,
                                        # Turn off colorbar
                                        add_colorbar=False)
# Add legend using earthpy
ep.draw_legend(im,
               titles=height_class_labels)
ax.set(title="Classified Lidar Canopy Height Model \n Derived from NEON AOP Data")
ax.set_axis_off()
plt.show()

## Part 4: Crop Raster Data

1. Import some more dependencies

In [ ]:
from shapely.geometry import mapping #https://shapely.readthedocs.io/en/stable/manual.html?highlight=mapping#shapely.geometry.mapping
import geopandas as gpd #https://geopandas.org/en/stable/index.html

2. Open the vector cropping layer and print CRS data

In [ ]:
aoi = os.path.join("spatial",
                   "boulder-leehill-rd",
                   "clip-extent.shp")

# Open crop extent (your study area extent boundary)
crop_extent = gpd.read_file(aoi) #https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html

print('crop extent crs: ', crop_extent.crs) #https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.crs.html
print('lidar crs: ', lidar_chm_xr.rio.crs)

In [ ]:
# Plot the crop boundary layer
# Note this is just an example so you can see what it looks like
# You don't need to plot this layer in your homework!
fig, ax = plt.subplots(figsize=(6, 6))

crop_extent.plot(ax=ax) #https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html

ax.set_title("Shapefile Crop Extent",
             fontsize=16)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
lidar_chm_xr.plot.imshow(ax=ax)

crop_extent.plot(ax=ax,
                 alpha=.8)
ax.set(title="Raster Layer with Shapefile Overlayed")

ax.set_axis_off()
plt.show()

3. Clip the CHM data to the vector boundary

In [ ]:
lidar_clipped = lidar_chm_xr.rio.clip(crop_extent.geometry.apply(mapping),
                                      # This is needed if your GDF is in a diff CRS than the raster data
                                      crop_extent.crs) #https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.raster_array.RasterArray.clip

f, ax = plt.subplots(figsize=(10, 4))
lidar_clipped.plot(ax=ax)
ax.set(title="Raster Layer Cropped to Geodataframe Extent")
ax.set_axis_off()
plt.show()

## Optional Extra credit (+5 points)
1. Generate a post-flood canopy height model
    1. You can use very similar commands to the pre-flood CHM but make sure you use the correct data path for post-flood data. Remember how we hard-coded the pre-flood path towards the top of the notebook? I recommend creating a new variable called `lidar_dem_path_post` to avoid variable conflicts and to use the `_post` suffix for all post-fire calculations. 
    ``` python
    lidar_dem_path = os.path.join("spatial",
                              "boulder-leehill-rd",
                              "pre-flood",
                              "lidar",
                              "pre_DTM.tif")
    ```
    becomes
    ``` python
    lidar_dem_path_post = os.path.join("spatial",
                              "boulder-leehill-rd",
                              "post-flood",
                              "lidar",
                              "post_DTM.tif")
    ```
    then duplicate the remaining Python commands, making sure to update variable names with `_post` as necessary.

    Please visualize your results too.

2. CHM-change raster
    1. Similar to how we calculated NBR change raster during a previous lab, calculate the change in CHM and plot results. 

    After you generate the post-fire CHM you can calculate the difference between pre- and post-fire.
    ``` python
    lidar_chm_xr_delta = lidar_chm_xr_post - lidar_chm_xr
    ```

    Please visualize the change in CHM.

## Part 5: Readings
Please read these articles then write 4-6 sentences about canopy height modeling and how it is similar or different across sensor platforms and geographic scales.
1. https://www.sciencedirect.com/science/article/am/pii/S0034425720305381 
1. https://www.mdpi.com/1999-4907/8/9/340 

Put your summary here


## Part 6: Submit lab

1. Click **File** > **Download** > **Download .iypnb**
2. Navigate to the repo you created during lab 0
3. Click **Add file** > **Upload files**
4. Click **choose your files** then navigate to your downloaded Lab 4 notebook then select it for upload
5. Click **Commit changes**

Reach out to your instructor on Discord, GitHub, or email if you run into any issues. 